A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render.

In [17]:
earning_yield = "EV/EBIT"
return_on_capital = "ROIC"

In [18]:
import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime, date
import numpy as np
import pandas as pd
# import yfinance as yf

def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))

dados do site https://www.fundamentus.com.br/resultado.php

In [19]:
import requests
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
dfunds = pd.DataFrame(funda)
dfunds.tail(5)

,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
Papel,,,,,,,,,,,,,,,,,,,,
CEPE6,43.00,534.72,2.03,0.379,0.0,0.233,9.84,2.63,-0.38,8.68,6.48,14.4,0.07,1.1,10.01,0.38,0.0,1.580000e+09,5.15,331.55
UBBR4,7.49,610.27,1.99,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.33,0.0,1.031720e+10,0.00,10.58
UBBR11,14.75,1201.81,3.91,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.33,0.0,1.031720e+10,0.00,10.58
UBBR3,18.00,1466.61,4.77,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.33,0.0,1.031720e+10,0.00,10.58
CEPE3,128.00,1591.72,6.04,1.127,0.0,0.695,29.30,7.83,-1.13,13.88,10.37,14.4,0.07,1.1,10.01,0.38,0.0,1.580000e+09,5.15,331.55


In [20]:
dfunds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

filtros e ranking

In [21]:
funds = dfunds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]

funds =  funds[funds['Liq.2meses'] > 300000] #Volume diário médio negociado nos últimos 2 meses
funds =  funds[funds['P/L'] > 0] 
# funds =  funds[(funds['P/L'] > 0) & (funds['P/L'] < 60)] 
funds =  funds[funds['Dív.Brut/ Patrim.'] < 4]
# funds =  funds[funds['Cresc. Rec.5a'] > 0]
# funds =  funds[funds['ROE'] > 0]
# funds =  funds[funds['EV/EBITDA'] > 0]


""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1
# print(dfunds.head(15).to_string())
funds

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,26.12,7.77,1.42,0.540,5.40,0.309,5.48,0.66,-2.62,...,1.09,59.64,18.33,4.273440e+07,1.185790e+10,0.00,15.92,1.0,1.0,2.0
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.268060e+09,3.860070e+11,0.79,20.27,7.0,15.0,22.0
3,WIZC3,6.26,10.29,2.18,0.953,7.11,0.436,-17.18,2.18,-1.21,...,0.90,24.37,21.18,2.815120e+06,4.592540e+08,0.77,11.69,5.0,18.0,23.0
4,PETR3,44.30,4.22,1.50,1.077,16.36,0.564,-79.60,2.40,-1.18,...,0.95,25.76,35.47,3.918260e+08,3.860070e+11,0.79,20.27,8.0,15.0,23.0
5,CAMB3,11.20,6.17,2.16,0.989,2.66,1.337,5.73,4.27,13.00,...,1.93,33.41,35.03,9.550330e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,MELK3,4.45,12.69,0.78,0.908,12.61,0.377,0.69,25.14,1.14,...,3.08,1.89,6.17,6.744230e+05,1.171940e+09,0.24,21.63,163.0,166.0,329.0
168,VSTE3,16.40,7.82,1.81,1.683,0.70,1.104,10.45,67.98,-16.79,...,1.48,1.81,23.13,6.040540e+05,1.028360e+09,0.30,7.34,169.0,167.0,336.0
169,EZTC3,15.36,18.04,0.74,3.214,1.33,0.581,1.61,60.08,2.36,...,5.19,1.13,4.09,2.857380e+07,4.598390e+09,0.18,10.57,168.0,168.0,336.0
170,ITSA3,10.43,7.80,1.35,14.528,5.27,1.048,21.63,118.64,-17.06,...,1.65,0.97,17.31,1.973110e+06,7.973800e+10,0.14,14.42,170.0,170.0,340.0


retira bancos

In [22]:
setor=20
url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
content = requests.get(url, headers=hdr)
df = pd.read_html(content.text, decimal=",", thousands='.')[0]
bancos = list(df['Papel'])

funds = funds[~funds['Papel'].isin(bancos)]
funds

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,26.12,7.77,1.42,0.540,5.40,0.309,5.48,0.66,-2.62,...,1.09,59.64,18.33,4.273440e+07,1.185790e+10,0.00,15.92,1.0,1.0,2.0
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.268060e+09,3.860070e+11,0.79,20.27,7.0,15.0,22.0
3,WIZC3,6.26,10.29,2.18,0.953,7.11,0.436,-17.18,2.18,-1.21,...,0.90,24.37,21.18,2.815120e+06,4.592540e+08,0.77,11.69,5.0,18.0,23.0
4,PETR3,44.30,4.22,1.50,1.077,16.36,0.564,-79.60,2.40,-1.18,...,0.95,25.76,35.47,3.918260e+08,3.860070e+11,0.79,20.27,8.0,15.0,23.0
5,CAMB3,11.20,6.17,2.16,0.989,2.66,1.337,5.73,4.27,13.00,...,1.93,33.41,35.03,9.550330e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,LOGG3,20.45,10.87,0.56,9.489,3.41,0.334,5.16,12.93,-1.31,...,1.77,2.79,5.15,5.642250e+06,3.729940e+09,0.51,16.33,164.0,161.0,325.0
166,NGRD3,1.08,27.82,0.56,0.952,1.37,0.384,1.55,53.58,3.66,...,2.37,1.11,2.03,4.465320e+05,4.564490e+08,0.11,7.00,157.0,169.0,326.0
167,MELK3,4.45,12.69,0.78,0.908,12.61,0.377,0.69,25.14,1.14,...,3.08,1.89,6.17,6.744230e+05,1.171940e+09,0.24,21.63,163.0,166.0,329.0
168,VSTE3,16.40,7.82,1.81,1.683,0.70,1.104,10.45,67.98,-16.79,...,1.48,1.81,23.13,6.040540e+05,1.028360e+09,0.30,7.34,169.0,167.0,336.0


retirar seguradoras

In [23]:
setor=31
url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
content = requests.get(url, headers=hdr)
df = pd.read_html(content.text, decimal=",", thousands='.')[0]
seguros = list(df['Papel'])

funds = funds[~funds['Papel'].isin(seguros)]
funds

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.268060e+09,3.860070e+11,0.79,20.27,7.0,15.0,22.0
4,PETR3,44.30,4.22,1.50,1.077,16.36,0.564,-79.60,2.40,-1.18,...,0.95,25.76,35.47,3.918260e+08,3.860070e+11,0.79,20.27,8.0,15.0,23.0
5,CAMB3,11.20,6.17,2.16,0.989,2.66,1.337,5.73,4.27,13.00,...,1.93,33.41,35.03,9.550330e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
6,BOBR4,2.32,5.63,-8.40,0.410,0.00,0.671,-7.01,2.81,-1.13,...,0.84,31.40,-149.20,2.141310e+05,-7.194700e+07,-5.84,8.52,18.0,9.0,27.0
7,VLID3,19.59,7.92,1.18,0.817,4.89,0.598,2.38,3.49,25.93,...,2.00,22.55,14.90,9.653640e+06,1.358760e+09,0.54,1.44,12.0,21.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,LOGG3,20.45,10.87,0.56,9.489,3.41,0.334,5.16,12.93,-1.31,...,1.77,2.79,5.15,5.642250e+06,3.729940e+09,0.51,16.33,164.0,161.0,325.0
166,NGRD3,1.08,27.82,0.56,0.952,1.37,0.384,1.55,53.58,3.66,...,2.37,1.11,2.03,4.465320e+05,4.564490e+08,0.11,7.00,157.0,169.0,326.0
167,MELK3,4.45,12.69,0.78,0.908,12.61,0.377,0.69,25.14,1.14,...,3.08,1.89,6.17,6.744230e+05,1.171940e+09,0.24,21.63,163.0,166.0,329.0
168,VSTE3,16.40,7.82,1.81,1.683,0.70,1.104,10.45,67.98,-16.79,...,1.48,1.81,23.13,6.040540e+05,1.028360e+09,0.30,7.34,169.0,167.0,336.0


retirar BDR

In [24]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds = funds[~funds['Papel'].str.contains(padrao)]
funds

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.268060e+09,3.860070e+11,0.79,20.27,7.0,15.0,22.0
4,PETR3,44.30,4.22,1.50,1.077,16.36,0.564,-79.60,2.40,-1.18,...,0.95,25.76,35.47,3.918260e+08,3.860070e+11,0.79,20.27,8.0,15.0,23.0
5,CAMB3,11.20,6.17,2.16,0.989,2.66,1.337,5.73,4.27,13.00,...,1.93,33.41,35.03,9.550330e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
6,BOBR4,2.32,5.63,-8.40,0.410,0.00,0.671,-7.01,2.81,-1.13,...,0.84,31.40,-149.20,2.141310e+05,-7.194700e+07,-5.84,8.52,18.0,9.0,27.0
7,VLID3,19.59,7.92,1.18,0.817,4.89,0.598,2.38,3.49,25.93,...,2.00,22.55,14.90,9.653640e+06,1.358760e+09,0.54,1.44,12.0,21.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,LOGG3,20.45,10.87,0.56,9.489,3.41,0.334,5.16,12.93,-1.31,...,1.77,2.79,5.15,5.642250e+06,3.729940e+09,0.51,16.33,164.0,161.0,325.0
166,NGRD3,1.08,27.82,0.56,0.952,1.37,0.384,1.55,53.58,3.66,...,2.37,1.11,2.03,4.465320e+05,4.564490e+08,0.11,7.00,157.0,169.0,326.0
167,MELK3,4.45,12.69,0.78,0.908,12.61,0.377,0.69,25.14,1.14,...,3.08,1.89,6.17,6.744230e+05,1.171940e+09,0.24,21.63,163.0,166.0,329.0
168,VSTE3,16.40,7.82,1.81,1.683,0.70,1.104,10.45,67.98,-16.79,...,1.48,1.81,23.13,6.040540e+05,1.028360e+09,0.30,7.34,169.0,167.0,336.0


In [25]:
funds.head(30)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
2,PETR4,42.69,4.07,1.44,1.038,16.98,0.543,-76.70,2.31,-1.13,...,0.95,25.76,35.47,1.268060e+09,3.860070e+11,0.79,20.27,7.0,15.0,22.0
4,PETR3,44.30,4.22,1.50,1.077,16.36,0.564,-79.60,2.40,-1.18,...,0.95,25.76,35.47,3.918260e+08,3.860070e+11,0.79,20.27,8.0,15.0,23.0
5,CAMB3,11.20,6.17,2.16,0.989,2.66,1.337,5.73,4.27,13.00,...,1.93,33.41,35.03,9.550330e+05,2.192080e+08,0.13,28.07,19.0,5.0,24.0
6,BOBR4,2.32,5.63,-8.40,0.410,0.00,0.671,-7.01,2.81,-1.13,...,0.84,31.40,-149.20,2.141310e+05,-7.194700e+07,-5.84,8.52,18.0,9.0,27.0
7,VLID3,19.59,7.92,1.18,0.817,4.89,0.598,2.38,3.49,25.93,...,2.00,22.55,14.90,9.653640e+06,1.358760e+09,0.54,1.44,12.0,21.0,33.0
8,KEPL3,9.34,6.35,2.46,1.110,9.20,1.215,4.46,5.50,7.36,...,1.68,33.03,38.76,1.085620e+07,6.816180e+08,0.31,33.78,34.0,6.0,40.0
9,UNIP3,64.25,8.61,2.38,1.173,4.92,1.077,5.25,5.22,-9.37,...,1.92,29.80,27.61,9.266320e+05,2.808340e+09,0.47,21.79,30.0,11.0,41.0
10,LEVE3,33.45,6.32,2.89,1.022,27.97,1.440,4.67,5.56,9.88,...,1.91,36.02,45.73,1.970970e+07,1.568530e+09,0.31,18.50,38.0,3.0,41.0
11,VALE3,67.68,6.47,1.62,1.489,8.98,0.689,59.63,3.84,-1.76,...,1.08,20.11,24.99,1.740900e+09,1.901720e+11,0.37,10.73,16.0,27.0,43.0
12,CMIN3,6.51,11.59,3.12,2.059,11.93,1.205,4.00,6.09,-7.58,...,2.96,33.73,26.92,4.803790e+07,1.144420e+10,0.74,-9.63,40.0,4.0,44.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.